## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,60.46,59,75,17.27,broken clouds
1,1,Hofn,IS,64.2539,-15.2082,27.00,67,0,16.11,clear sky
2,2,Thompson,CA,55.7435,-97.8558,-11.04,64,20,4.61,few clouds
3,3,Sorong,ID,-0.8833,131.2500,80.29,81,100,11.25,overcast clouds
4,4,Mar Del Plata,AR,-38.0023,-57.5575,66.22,89,100,24.00,overcast clouds


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Sorong,ID,-0.8833,131.2500,80.29,81,100,11.25,overcast clouds
6,6,Vanimo,PG,-2.6741,141.3028,77.63,87,100,5.12,overcast clouds
10,10,Olinda,BR,-8.0089,-34.8553,85.73,54,40,9.22,scattered clouds
11,11,Antofagasta,CL,-23.6500,-70.4000,73.22,60,0,7.85,clear sky
13,13,Vao,NC,-22.6667,167.4833,79.95,84,84,1.05,broken clouds
14,14,Avarua,CK,-21.2078,-159.7750,77.05,88,85,10.36,overcast clouds
17,17,Canete,CL,-37.8000,-73.4000,74.66,44,0,8.39,clear sky
21,21,Hithadhoo,MV,-0.6000,73.0833,82.47,73,32,12.55,scattered clouds
25,25,Rikitea,PF,-23.1203,-134.9692,79.09,70,2,16.75,clear sky
30,30,Kalemie,CD,-5.9475,29.1947,70.43,89,100,5.57,light rain


In [16]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                260
City                   260
Country                255
Lat                    260
Lng                    260
Max Temp               260
Humidity               260
Cloudiness             260
Wind Speed             260
Current Description    260
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID                255
City                   255
Country                255
Lat                    255
Lng                    255
Max Temp               255
Humidity               255
Cloudiness             255
Wind Speed             255
Current Description    255
dtype: int64

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Sorong,ID,80.29,overcast clouds,-0.8833,131.2500,
6,Vanimo,PG,77.63,overcast clouds,-2.6741,141.3028,
10,Olinda,BR,85.73,scattered clouds,-8.0089,-34.8553,
11,Antofagasta,CL,73.22,clear sky,-23.6500,-70.4000,
13,Vao,NC,79.95,broken clouds,-22.6667,167.4833,
14,Avarua,CK,77.05,overcast clouds,-21.2078,-159.7750,
17,Canete,CL,74.66,clear sky,-37.8000,-73.4000,
21,Hithadhoo,MV,82.47,scattered clouds,-0.6000,73.0833,
25,Rikitea,PF,79.09,clear sky,-23.1203,-134.9692,
30,Kalemie,CD,70.43,light rain,-5.9475,29.1947,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
     # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Sorong,ID,80.29,overcast clouds,-0.8833,131.2500,Waigo Splash Hotel
6,Vanimo,PG,77.63,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
10,Olinda,BR,85.73,scattered clouds,-8.0089,-34.8553,Hotel Albergue de Olinda
11,Antofagasta,CL,73.22,clear sky,-23.6500,-70.4000,"Holiday Inn Express Antofagasta, an IHG Hotel"
13,Vao,NC,79.95,broken clouds,-22.6667,167.4833,Hotel Kou-Bugny
...,...,...,...,...,...,...,...
699,Manuk Mangkaw,PH,80.31,scattered clouds,4.8000,119.8500,
703,Buala,SB,80.69,light rain,-8.1450,159.5921,Maringe Lagoon Lodge
704,Clarence Town,AU,70.86,few clouds,-32.5833,151.7833,River Downs Cottage
708,Mudon,MM,73.89,overcast clouds,16.2578,97.7164,Motel Mudon


In [27]:
# 8a. Create the output File (CSV)
clean_hotel_df=hotel_df.copy()
output_data_file = "../World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Descriptio</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))